In [1]:
!pip install --quiet -U gensim

     |████████████████████████████████| 24.1 MB 9.3 MB/s 


In [2]:
!pip install --quiet transformers

     |████████████████████████████████| 4.7 MB 8.3 MB/s 
     |████████████████████████████████| 120 kB 78.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.1 MB/s 


In [3]:
!pip3 install --quiet keras

In [4]:
!pip3 install --quiet keras

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd drive/MyDrive/AML/HW4/classifier

/content/drive/MyDrive/AML/HW4/classifier


In [7]:
!ls

 BERT-classification-on-text-augmented.ipynb
 BERT-classification-on-text.ipynb
 BERT-pretrained-features.ipynb
 classification_test_results.tsv
 dreaddit
 dreaddit-test.csv
 dreaddit-train.csv
 dreddit.ipynb
 GoogleNews-vectors-negative300.bin
 GoogleNews-vectors-negative300.bin.gz
 logs
 pre-trained-BERT-features-2.ipynb
 results
 tBERT_LDA_augmented_dreddit.ipynb
 tBERT_with_LDA_for_dreddit.ipynb
 test_data.tsv
 tfi-df-embeddings.ipynb
 train_data_for_students.tsv
 trump_tweets_classifiers.ipynb
'w2v_domain_trained_ GSDMM.ipynb'
 w2v-domain-trained.ipynb
 w2v_domain_trained_lda_augmented_self_training.ipynb
 w2v-domain-trained-lda.ipynb
 w2v_domain_trained_LDA_multilabled.ipynb
 w2v-google-news-pretrained.ipynb


In [8]:
# Data processing
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer

import transformers
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

# Visualization
from matplotlib import pyplot as plt

import nltk
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder

# Text processing
import re
import nltk
import gensim.downloader as api

# Various
from datetime import datetime
#Google Drive
from google.colab import drive

import re
import tqdm

import gensim, logging


In [9]:
def set_seed(seed: int):
    # numpy
    np.random.seed(seed)

    # torch
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
         
set_seed(42)

In [10]:
def concat_two(original, new_x, features_from_original):
  new_df = pd.concat([new_x, original[features_from_original]], axis=1)
  return new_df

In [11]:
def get_domain(subreddit):
  if subreddit=='domesticviolence' or subreddit=='survivorsofabuse':
    return 'abuse';
  if subreddit=='anxiety' or subreddit=='stress':
    return 'anxiety';
  if subreddit=='almosthomeless' or subreddit=='assistance' or subreddit=='food_pantry' or subreddit=='homeless':
    return 'financial';
  if subreddit=='ptsd':
    return 'ptsd';
  else:
    return 'social';

In [12]:
df=pd.read_csv('dreaddit/dreaddit-train.csv')
df.shape

(2838, 116)

In [13]:
df = df[df['confidence'] > 0.79]

In [14]:
df_test = pd.read_csv('dreaddit/dreaddit-test.csv')
df_test.shape

(715, 116)

In [15]:
df = df.reset_index()
df_test = df_test.reset_index()

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
# Reference: Text Classification with NLP: Tf-Idf vs Word2Vec vs BERT from Medium

# Use stopwords list from nltk
lst_stopwords = nltk.corpus.stopwords.words("english")

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    # Clean (convert to lowercase and remove punctuations and characters and then strip)
    # The function is not optimized for speed but split into various steps for pedagogical purpose
    text = str(text).lower()
    text = text.strip()
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize (convert from string to list)
    lst_text = text.split()
    # remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    # Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    # Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    # back to string from list
    text = " ".join(lst_text)
    return text


In [18]:
df['text_clean'] = df["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=lst_stopwords))
df['domain'] = df['subreddit'].map(lambda x: get_domain(x))
X_train = df['text_clean']
y_train = df['label']
y_train = y_train.to_list()
df.head(2)

,index,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,text_clean,domain
0,0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,...,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742,said felt way suggeted go rest trigger ahead y...,ptsd
1,1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,...,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857,hey rassistance sure right place post goes im ...,financial


In [19]:
df_test['text_clean'] = df["text"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=lst_stopwords))
df_test['domain'] = df_test['subreddit'].map(lambda x: get_domain(x))
X_test = df_test['text_clean']
y_test = df_test['label']
y_test = y_test.to_list()
df_test.head(2)

,index,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,...,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment,text_clean,domain
0,0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not.“ ME: I have...",0,0.8,1.514981e+09,22,...,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000,said felt way suggeted go rest trigger ahead y...,social
1,1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1.493348e+09,5,...,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909,hey rassistance sure right place post goes im ...,anxiety


In [20]:
ll = df[df.columns[1:]].corr().abs()['label'].sort_values(ascending = False)[:16].gt(0.2).drop('label')

In [21]:
top_correlations = list(ll.index)
top_correlations

['lex_liwc_Tone',
 'lex_liwc_Clout',
 'lex_liwc_negemo',
 'lex_liwc_i',
 'sentiment',
 'lex_dal_min_pleasantness',
 'lex_liwc_posemo',
 'lex_liwc_anx',
 'lex_liwc_Authentic',
 'lex_liwc_Analytic',
 'lex_liwc_social',
 'lex_liwc_function',
 'lex_liwc_Dic',
 'lex_liwc_anger',
 'lex_liwc_you']

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [23]:
class BERT_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [24]:
class BertTrainData:
  def __init__(self, X_train, X_test, y_train, y_test):
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

    self.train_texts = X_train
    self.valid_texts = X_val
    self.train_labels = y_train
    self.valid_labels = y_val
    self.test_text = X_test
    self.test_labels = y_test

In [25]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=20,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    load_best_model_at_end=True,    
    logging_steps=250,              
    evaluation_strategy="steps",
);

In [26]:
def get_prediction(text, max_length, targets, tokenizer, model):
    # tokenize the input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # get output of the fine-tuned model for this tokenized input
    outputs = model(**inputs)
    # convert model outputs to probabilities by performing a softmax
    probs = outputs[0].softmax(1)
    # use arg max to find the index where probabilty is the maximum, this index is our predicted label for the input text
    return targets[probs.argmax()]

def tain_bert_model(model_name, targets, max_length, train_data, do_lower_case):

    tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=do_lower_case)
    train_encodings = tokenizer(train_data.train_texts, truncation=True, padding=True, max_length=max_length)
    valid_encodings = tokenizer(train_data.valid_texts, truncation=True, padding=True, max_length=max_length)

    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(targets)).to("cuda")

    trainer = Trainer(
        model=model,                        
        args=training_args,                  
        train_dataset=BERT_Dataset(train_encodings, train_data.train_labels),        
        eval_dataset=BERT_Dataset(valid_encodings, train_data.valid_labels),          
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return model, tokenizer, trainer

In [27]:
test_size = 0.1
max_length = 512
targets = ['NoStressed','Stressed']

In [28]:
X_train = df['text']
y_train = df['label']
X_test = df_test['text']
y_test = df_test['label']

In [29]:
pretrained_model_name = "bert-base-uncased"

train_data = BertTrainData(X_train.tolist(), X_test.tolist(), y_train.tolist(), y_test.tolist())

model, tokenizer, trainer = tain_bert_model(pretrained_model_name, targets, max_length, train_data, False)

evaluation_result = trainer.evaluate()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy
250,0.406700,0.304387,0.898488


***** Running Evaluation *****
  Num examples = 463
  Batch size = 20


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 463
  Batch size = 20


In [30]:
print(evaluation_result)

{'eval_loss': 0.2938348352909088, 'eval_accuracy': 0.896328293736501, 'eval_runtime': 4.647, 'eval_samples_per_second': 99.634, 'eval_steps_per_second': 5.165, 'epoch': 3.0}


In [31]:
predicted_labels_bert_cased = []
for instance in train_data.test_text:
  predicted_label = get_prediction(instance, max_length, targets, tokenizer, model)
  predicted_labels_bert_cased.append(predicted_label)
true_labels = [targets[y] for y in train_data.test_labels]
accuracy_bert_cased = accuracy_score(true_labels, predicted_labels_bert_cased)

print(classification_report(true_labels, predicted_labels_bert_cased))

              precision    recall  f1-score   support

  NoStressed       0.80      0.77      0.78       346
    Stressed       0.79      0.82      0.80       369

    accuracy                           0.79       715
   macro avg       0.79      0.79      0.79       715
weighted avg       0.79      0.79      0.79       715



In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_labels, predicted_labels_bert_cased)

array([[266,  80],
       [ 67, 302]])